This code identifies potential missing connections in a road network by detecting node pairs that are either geographically close (within 300 meters) or have an excessively long network path relative to their straight-line distance (more than 5 times longer).

The hyperparameters of the method.

In [ ]:
# Method generating new edges which are connecting geographically far nodes

import osmnx as ox
import networkx as nx
from itertools import combinations

# Load a graph (e.g., Rome, Italy) ...

G = ox.graph_from_place('Amurzet_town, Russia', network_type="drive", simplify=True) #("Bamut_village, Russia", network_type="drive", simplify=True)
G_undir = G.to_undirected()

#  Generate candidate edges from nodes which are close geographically
candidate_edges = []
for u, v in combinations(G.nodes(), 2):
    if not G.has_edge(u, v):
        u_coord = (G.nodes[u]['y'], G.nodes[u]['x'])
        v_coord = (G.nodes[v]['y'], G.nodes[v]['x'])
        euclidean_dist = ox.distance.great_circle(G.nodes[u]['y'], G.nodes[u]['x'], G.nodes[v]['y'], G.nodes[v]['x'])
        #ox.distance.great_circle()
        if euclidean_dist < 300:  # 300-meter threshold
            candidate_edges.append((u, v))

            if not G.has_edge(u, v):
                try:
                    network_dist = nx.shortest_path_length(G, u, v, weight='length')
                    if network_dist / euclidean_dist > 5: # if network distance is much larger than geographical distance
                      candidate_edges.append((u, v))
                except nx.NetworkXNoPath:
                    continue

# Now we need to check whether we add anything from those nodes
print(candidate_edges)

In [ ]:
# plotting the graph which are added in the graph and checking the connectivity levels
import matplotlib as plt


# Plot base graph with gray edges
fig, ax = ox.plot_graph(
    G,
    show=False,
    close=False,
    bgcolor="white",
    edge_color="gray",
    node_size=0,
    edge_linewidth=0.5
)

# Overlay candidate edges in red
for u, v in candidate_edges:
    if u in G.nodes() and v in G.nodes():  # Ensure nodes exist
        xu, yu = G.nodes[u]["x"], G.nodes[u]["y"]
        xv, yv = G.nodes[v]["x"], G.nodes[v]["y"]
        ax.plot(
            [xu, xv],
            [yu, yv],
            color="red",
            linestyle="--",
            linewidth=2,
            alpha=0.7,
            zorder=3  # Ensure candidate edges appear above base graph
        )

plt.title("Network with Suggested Links (Red Dashes)")
plt.show()


